# Implementing a Spam Filter with Naive Bayes Algorithm from scratch
We shall train a **`Naive Bayes` classification model** which is based on  **multinomial Naive Bayes machine learning algorithm**. Then use the model to classify new messages as either **spam** or **non spam** or **ham**.

**Our goal is to create a spam filter that classifies new messages with an accuracy greater than `80%`.** 

## The Dataset
As training dataset, we shall use a dataset of **`5,572` SMS messages** that are already classified or labelled by humans. The dataset was put together by Tiago **A. Almeida** and **José María Gómez Hidalgo**, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) or from [Amazon](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). 

The data collection process is described in more details in the [authors papers](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('./data/SMSSpamCollection', sep = '\t', header = None, names = ['Label', 'SMS'])
print(data.shape)
data.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Let's find the distribution of `spam` Vs `ham` in the dataset

In [3]:
label_count = data.Label.value_counts()

label_count_df = pd.DataFrame.from_dict(dict(label_count), orient = 'index')
label_count_df.rename(columns = {0: 'frequency'}, inplace = True)
label_count_df.sort_values(by = 'frequency', ascending = False, inplace = True)
label_count_df['percentage'] = label_count_df.frequency.apply(lambda x: (100 * x / label_count_df.frequency.sum()))
label_count_df.head()

,frequency,percentage
ham,4825,86.593683
spam,747,13.406317


We can see that we are dealing hugely **imbalance dataset!** We have about **`87%` of non-spam** with only **`13%` of the dataset labelled as spam**. This poses a serious problem because the **model would tend to be bias towards non-spam** messages. We shall have to address the problem posed by imbalance data before training the model.

## Training the model
### Training and Test Set
We're going to use **`80%`** of our dataset (`4,458` messages) for training, and **`20%`** (`1,114` messages) for testing. Here, we shall use `train_test_split` function from `scikit-learn` to split the dataset.

In [4]:
X = data.drop('Label', axis = 1)   # features / messages
y = data.Label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

#### Let's find the distribution of spam Vs ham in both training and test datasets

In [5]:
y_train.value_counts(normalize = True) * 100

Label
ham     86.53803
spam    13.46197
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize = True) * 100

Label
ham     86.816143
spam    13.183857
Name: proportion, dtype: float64

**We can see that the function `train_test_split(test_size = 0.20)` split the dataset as expected while preserving the proportions of `ham` vs `spam` in the original dataset.**

### Letter Case and Punctuation
The **Naive Bayes** algorithm will make the classification based on the probability that the message is **spam** or **not spam (ham)**. These probabilities are calculated as follows:

$$P(Spam | w_1, w_2, ... w_n) \propto P(Spam).\prod\limits_{i=1}^{n}P(w_i | Spam)$$

$$P(Ham | w_1, w_2, ... w_n) \propto P(Ham).\prod\limits_{i=1}^{n}P(w_i | Ham)$$

To calculate $P(w_i | Spam)$ and $P(w_i | Ham)$ inside the formulas above, we need to use these equations:

$$P(w_i | Spam) = \frac{N_{w_{i|Spam}} + \alpha}{N_{Spam} + \alpha . N_{vocab}}$$

$$P(w_i | Ham) = \frac{N_{w_{i|Ham}} + \alpha}{N_{Ham} + \alpha . N_{vocab}}$$

Where:

* $N_{w_{i|Spam}}$ = the number of times the word $w_i$ occurs in spam messages

* $N_{w_{i|Ham}}$ = the number of times the word $w_i$ occurs in non-spam messages

* $N_{Spam}$ = total number of words in spam messages

* $N_{Ham}$ = total number of words in non-spam messages

* $N_{vocab}$ = total number of words in the vocabulary

* $\alpha = 1$ is a smoothing parameter

It's worth noting that these calculations make the naive assumption that **words in the text messages are statistically independent**. However, this is not true in real life. Words take context meaning in sentences. This context meaning is considered in **Natural Language Processing (NLP)** applications that use **word embeddings** like **Large Language Models (LLMs)**. An example of LLM application is **ChatGPT**. 

This ***naive*** assumption in the implementation of the algorithm leads to its name as **Naive Bayes**.

### Creating the Vocabulary for the features
The **vocabulary** is a set of **unique** words in the dataset.
* let's create a list with all of the unique words that occur in the messages of our training set.

### Data Cleaning
Since computers can only process numerical data, we will have to transform the data to numerical format before training the model.

* Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case.
* Then transform every letter in every word to lower case.

In [7]:
import warnings
warnings.filterwarnings('ignore')

X_train.loc[:, 'SMS'] = X_train.SMS.apply(lambda x: re.sub('\W', ' ', x))
X_train.loc[:, 'SMS'] = X_train.SMS.str.lower()
X_train.head()

,SMS
1642,hi where are you we re at and they re not ...
2899,if you r home then come down within 5 min
480,when re you guys getting back g said you were...
3485,tell my bad character which u dnt lik in me ...
157,i m leaving my house now


In [8]:
X_train.loc[:, 'SMS'] = X_train.SMS.str.split()
X_train.head()

,SMS
1642,"[hi, where, are, you, we, re, at, and, they, r..."
2899,"[if, you, r, home, then, come, down, within, 5..."
480,"[when, re, you, guys, getting, back, g, said, ..."
3485,"[tell, my, bad, character, which, u, dnt, lik,..."
157,"[i, m, leaving, my, house, now]"


In [9]:
# keep only word with more than 1 characters in vocabs
vocabulary = [word for sms in X_train.SMS for word in sms if len(word) > 1]
print('Length of vocabs list:', len(vocabulary))
vocabulary = list(set(vocabulary))
print('Length of unique vocabs:', len(vocabulary))

Length of vocabs list: 64063
Length of unique vocabs: 7714


### The Final Training Set

In [10]:
word_counts_per_sms = {word: [0] * len(X_train.SMS) for word in vocabulary}

for idx, sms in enumerate(X_train.SMS):
    for word in sms:
        if word in vocabulary:
            word_counts_per_sms[word][idx] += 1

In [11]:
#word_counts_df = pd.DataFrame(word_counts_per_sms)
word_counts_df = pd.DataFrame.from_dict(word_counts_per_sms, orient = 'index').T
word_counts_df.head()

,affection,letters,monkeespeople,ultimate,snap,08717898035,9t,epsilon,subs,pod,...,transaction,sac,laden,23g,married,watched,abdomen,shudvetold,80182,irritation
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating Constants First
Let's calculate the constant needed in the above probability equations. For example:

* the probability of a message being a Spam is simply the number of Spams divided by the total number of messages.

* the probability of a message not being a Spam is simply the number of Hams divided by the total number of messages.

#### Using the training set only

In [12]:
N_messages = len(word_counts_df)    # len(X_train)
N_vocab = len(vocabulary)
alpha = 1

In [13]:
n_spam = y_train.value_counts()['spam']        # number of spam messages
n_ham = y_train.value_counts()['ham']          # number of ham messages
P_spam = n_spam / N_messages                   # prior prob of spam
P_ham = n_ham / N_messages                     # prior prob of ham

print(P_spam + P_ham)

1.0


### Calculating Parameters
The probability values that $P(w_i | Spam)$ and $P(w_i | Ham)$ will take are called **parameters**.

Our vocabulary ffor the training dataset is of length **`7,753`** and for each word we need to calculate $P(w_i | Spam)$ for **Spam** and $P(w_i | Ham)$ for **non-Spam**. This means we shall have to calculated **`7,753 x 2 = 15,506` probabilities!!!** 

The fact that we calculate so many values (as constants) before even beginning the classification of new messages makes the **Naive Bayes algorithm very fast** (especially compared to other algorithms).

### Isolating the spam and the ham messages in the training set into two different DataFrames
To make our job easier going forward, we shall include the training Labels to our training features. We shall also include the word counts features into the DataFrame.

In [14]:
X_train['Label'] = y_train

In [15]:
X_train = pd.concat([X_train, word_counts_df], axis = 1)
X_train.head()

,SMS,Label,affection,letters,monkeespeople,ultimate,snap,08717898035,9t,epsilon,...,transaction,sac,laden,23g,married,watched,abdomen,shudvetold,80182,irritation
1642,"[hi, where, are, you, we, re, at, and, they, r...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2899,"[if, you, r, home, then, come, down, within, 5...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
480,"[when, re, you, guys, getting, back, g, said, ...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3485,"[tell, my, bad, character, which, u, dnt, lik,...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,"[i, m, leaving, my, house, now]",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Isolating the `spam` and the `ham` messages

In [16]:
spam_df = X_train.loc[X_train.Label == 'spam']
spam_df.head()

,SMS,Label,affection,letters,monkeespeople,ultimate,snap,08717898035,9t,epsilon,...,transaction,sac,laden,23g,married,watched,abdomen,shudvetold,80182,irritation
5365,"[camera, you, are, awarded, a, sipix, digital,...",spam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1625,"[500, free, text, msgs, just, text, ok, to, 80...",spam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4845,"[you, have, won, as, a, valued, vodafone, cust...",spam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3564,"[auction, round, 4, the, highest, bid, is, now...",spam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3998,"[bored, housewives, chat, n, date, now, 087175...",spam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
ham_df = X_train.loc[X_train.Label == 'ham']
ham_df.head()

,SMS,Label,affection,letters,monkeespeople,ultimate,snap,08717898035,9t,epsilon,...,transaction,sac,laden,23g,married,watched,abdomen,shudvetold,80182,irritation
1642,"[hi, where, are, you, we, re, at, and, they, r...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2899,"[if, you, r, home, then, come, down, within, 5...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
480,"[when, re, you, guys, getting, back, g, said, ...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3485,"[tell, my, bad, character, which, u, dnt, lik,...",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,"[i, m, leaving, my, house, now]",ham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Initialize two dictionaries that will hold the word counts used to calculate the parameters
#### Counting the unique words and updating the dictionaries

In [18]:
spam_dict = {}
ham_dict = {}

for sms in spam_df.SMS:
    for word in sms:
        if word not in spam_dict:
            spam_dict[word] = 0
        spam_dict[word] += 1

#### Number of words in spam messages

In [19]:
#N_spam = sum(spam_dict.values())
N_w_spam = sum(value for key, value in spam_dict.items() if len(key) > 1)  # keep only word with more than 1 characters

In [20]:
for sms in ham_df.SMS:
    for word in sms:
        if word not in ham_dict:
            ham_dict[word] = 0
        ham_dict[word] += 1

#### Number of words in ham messages

In [21]:
#N_ham = sum(ham_dict.values())
N_w_ham = sum(value for key, value in ham_dict.items() if len(key) > 1)    # keep only word with more than 1 characters

### Calculating the parameters

In [22]:
p_wi_spam = {}
p_wi_ham = {}

for word in vocabulary:
    p_wi_spam[word] = (spam_df[word].sum() + alpha)/(N_w_spam + alpha * N_vocab)
    p_wi_ham[word] = (ham_df[word].sum() + alpha)/(N_w_ham + alpha * N_vocab)

## Classifying A New Message
Helper function to classify messages as either **spam** or **ham**.

In [23]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = P_spam
    p_ham_given_message = P_ham
    
    for word in message:
        if word in p_wi_spam:
            p_spam_given_message *= p_wi_spam[word]
        if word in p_wi_ham:
            p_ham_given_message *= p_wi_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_spam_given_message > p_ham_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities, have a human classify this!')

### Putting our function to work

In [24]:
message = 'WINNER!! This is the secret code to unlock the money: C3421.'
classify(message)

P(Spam|message): 7.620825602825606e-29
P(Ham|message): 4.775405553037237e-25
Label: Ham


In [25]:
message = "Sounds good, Tom, then see u there"
classify(message)

P(Spam|message): 6.912318244205037e-23
P(Ham|message): 2.060362529863275e-19
Label: Ham


### Measuring the Spam Filter's Accuracy using the test dataset
We are going to use the **`1,115`** messages we kept aside as testing dataset to evaluate out classifier. Every message in the test set is practically new from the perspective of the algorithm since the algorithm didn't see these during training.

But we'll have to modify out classifier to output the predictions (spam or ham) alongside the true labels.

In [26]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = P_spam
    p_ham_given_message = P_ham
    
    for word in message:
        if word in p_wi_spam:
            p_spam_given_message *= p_wi_spam[word]
        if word in p_wi_ham:
            p_ham_given_message *= p_wi_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'undecided'

In [27]:
X_test['Label'] = y_test
X_test.head()

,SMS,Label
1078,"Yep, by the pretty sculpture",ham
4028,"Yes, princess. Are you going to make me moan?",ham
958,Welp apparently he retired,ham
4642,Havent.,ham
4674,I forgot 2 ask ü all smth.. There's a card on ...,ham


In [28]:
X_test['Prediction'] = X_test.SMS.apply(classify_test_set)
X_test.head()

,SMS,Label,Prediction
1078,"Yep, by the pretty sculpture",ham,ham
4028,"Yes, princess. Are you going to make me moan?",ham,ham
958,Welp apparently he retired,ham,ham
4642,Havent.,ham,ham
4674,I forgot 2 ask ü all smth.. There's a card on ...,ham,ham


### Calculating the Accuracy of the Classifier
The accuracy is calculated as follows:

$$Accuracy = \frac{number\space of\space  correctly\space  classified\space  messages}{total\space  number\space  of\space  classified\space  messages}$$

In [29]:
num_correct = sum(X_test['Prediction'] == X_test['Label'])
acc = num_correct / len(X_test)
print(f'The classifier got {num_correct} out of {len(X_test)} correct')
print(f'The classifier has an accuracy of {acc * 100: .2f}%')

The classifier got 972 out of 1115 correct
The classifier has an accuracy of  87.17%


**Wow! This looks great! Our goal was to obtain an accuracy of `80%` but we got `87%`! Our classifier is doing a great job!**

#### The Constants used

In [30]:
print(f'Prior Probability of getting a spam (P_spam): {P_spam: .2f}')
print(f'Prior Probability of not getting a spam (P_ham): {P_ham: .2f}')
print(f'Number of words in spam messages in dataset (N_w_spam): {N_w_spam: ,}')
print(f'Number of words in non spam messages in dataset (N_w_ham): {N_w_ham: ,}')
print(f'Number of unique words in vocabulary (N_vocab): {N_vocab: ,}')
print(f'Smoothing parameter (alpha): ', alpha)

Prior Probability of getting a spam (P_spam):  0.13
Prior Probability of not getting a spam (P_ham):  0.87
Number of words in spam messages in dataset (N_w_spam):  14,012
Number of words in non spam messages in dataset (N_w_ham):  50,051
Number of unique words in vocabulary (N_vocab):  7,714
Smoothing parameter (alpha):  1
